# Interactive Session 4
This Jupyter notebook is provided as a companion to Interactive Session 4, in order to practice what you have learned, and to present some new class material. It will help you to answer quiz questions included in this Interactive Session, and will give you an opportunity to experiment before answering the questions.

Please note that not all queries in the cells of this notebook are supposed to run properly. Some of them will fail and you are expected to find the reason for that.

## Initial Steps

Please run the following few cells before we start. They create the required tables and insert some tuples so that we can start experimenting with them.

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///is4.db

In [3]:
%%sql
CREATE TABLE Students (
    sid CHAR(11),
    name CHAR(20) NOT NULL,
    school CHAR(10),
    age INTEGER,
    gpa REAL,
    PRIMARY KEY (sid)
);

INSERT INTO students(sid, name, school, age, gpa)
VALUES  ('1001', 'Aadm', 'SFU', 23, 3.2),
        ('1002', 'Aiden', 'UBC', 19, 3.5),
        ('1003', 'Alice', 'SFU', 18, 3.7),
        ('1004', 'Bob', 'UBC', 22, 3.1),
        ('1005', 'David', 'SFU', 20, 3.2),
        ('1006', 'John', 'SFU', 21, 3.1),
        ('1007', 'Mary', 'UBC', 21, 3.4),
        ('1008', 'Mike', 'SFU', 24, 3.1),
        ('1009', 'Sarah', 'UBC', 18, 3.0);

SELECT * FROM students;

 * sqlite:///is4.db
Done.
9 rows affected.
Done.


sid,name,school,age,gpa
1001,Aadm,SFU,23,3.2
1002,Aiden,UBC,19,3.5
1003,Alice,SFU,18,3.7
1004,Bob,UBC,22,3.1
1005,David,SFU,20,3.2
1006,John,SFU,21,3.1
1007,Mary,UBC,21,3.4
1008,Mike,SFU,24,3.1
1009,Sarah,UBC,18,3.0


In [4]:
%%sql
CREATE TABLE Enrolled(
    stid CHAR(20),
    cid CHAR(20),
    grade CHAR(5),
    PRIMARY KEY (stid, cid),
    FOREIGN KEY (stid) REFERENCES Students(sid)
);

INSERT INTO Enrolled(stid, cid, grade)
VALUES  ('1001', '200','A'),
        ('1001', '295','A'),
        ('1001', '250','B+'),
        ('1002', '130','A'),
        ('1002', '125','A+'),
        ('1003', '120','A'),
        ('1003', '125','B'),
        ('1003', '150','A');

SELECT * FROM Enrolled;

 * sqlite:///is4.db
Done.
8 rows affected.
Done.


stid,cid,grade
1001,200,A
1001,295,A
1001,250,B+
1002,130,A
1002,125,A+
1003,120,A
1003,125,B
1003,150,A


In [5]:
%%sql
CREATE TABLE Movies(
    title        CHAR(100),
    year         INT,
    length       INT,
    genre        CHAR(10),
    studioName   CHAR(30),
    producerCNum INT,
    PRIMARY KEY  (title, year)
);

 * sqlite:///is4.db
(sqlite3.OperationalError) table Movies already exists
[SQL: CREATE TABLE Movies(
    title        CHAR(100),
    year         INT,
    length       INT,
    genre        CHAR(10),
    studioName   CHAR(30),
    producerCNum INT,
    PRIMARY KEY  (title, year)
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [6]:
%%sql
CREATE TABLE StarsIn(
       movieTitle CHAR(100), 
       movieYear  INT, 
       starName   CHAR(100)
);  

 * sqlite:///is4.db
Done.


[]

In [7]:
%%sql
CREATE TABLE MovieExec(
       name  CHAR(100), 
       address CHAR(100), 
       certNum INT, 
       netWorth INT
);

 * sqlite:///is4.db
Done.


[]

## Subqueries

### WHERE Clause Subqueries

In [3]:
%%sql
SELECT name 
FROM MovieExec 
WHERE certNum =
    ( SELECT producerCNum
      FROM Movies
      WHERE title = 'Star Wars' 
    );

 * sqlite:///is4.db
Done.


name


In [4]:
%%sql
SELECT name 
FROM MovieExec 
WHERE certNum IN 
    (   SELECT producerCNum
        FROM Movies
        WHERE (title, year) IN 
        (
            SELECT movieTitle, movieYear 
            FROM StarsIn
            WHERE starName = 'Harrison Ford'
        ) 
    ); 

 * sqlite:///is4.db
Done.


name


The following subquery does not work. Try to figure out what it is trying to do, and what is the reason it did not run. Can you rewrite it to make it work? 

In [ ]:
%%sql
SELECT title 
FROM Movies Old 
WHERE year > ANY 
    (    SELECT year
         FROM Movies
         WHERE title = Old.title 
    );

The answer is as follows (using MIN to rewrite and eliminate the use of using ANY). The reason the previous query did not run was the usage of ANY which is not supported in SQLite.

In [ ]:
%%sql
SELECT Old.title 
FROM Movies Old 
WHERE year >
    (    SELECT MIN(year)
         FROM Movies M2
         WHERE title = Old.title 
    );

### FROM Clause Subqueries

In [ ]:
%%sql
SELECT name
FROM MovieExec, 
    (    SELECT producerCNum
         FROM Movies, StarsIn
         WHERE title = movieTitle AND
         year = movieYear AND starName = 'Harrison Ford' 
    ) Prod 
WHERE certNum=Prod.producerCNum;

### HAVING Clause Subqueries

In [ ]:
%%sql
SELECT AVG(gpa), age
FROM Students 
WHERE gpa > 2.5
GROUP BY age
HAVING gpa >= (SELECT AVG(gpa) FROM Students);

## Constraint, Check, Assertion
What do you think about the next query?

In [ ]:
%%sql
DROP TABLE MovieStar;

In [ ]:
%%sql
CREATE TABLE MovieStar ( 
    name      CHAR(30) PRIMARY KEY, 
    lastname  CHAR(30) UNIQUE,
    address   VARCHAR(255) NOT NULL, 
    gender    CHAR(1),
    birthdate DATE, 
    CHECK (gender ='F' OR gender='M')
  );

Try different insertions below:

In [ ]:
%%sql
INSERT INTO MovieStar(name, lastname, address, gender, birthdate)
VALUES  ('Adam', 'Smith', '123 ABC Street', 'M', 1999-01-01)

Why each of the following inserts are being rejected?

In [ ]:
%%sql
INSERT INTO MovieStar(name, lastname, gender, birthdate)
VALUES  ('Mike', 'Moore','M', 1999-01-01)

In [ ]:
%%sql
INSERT INTO MovieStar(name, lastname, address, gender, birthdate)
VALUES  ('Michael', 'Smith', '123 ABC Street', 'F', 1999-01-01)

In [ ]:
%%sql
INSERT INTO MovieStar(name, lastname, address, gender, birthdate)
VALUES  ('Michael', 'ABC', '123 ABC Street', 'A', 1999-01-01)

Now, let's create the same condition using assertions.

In [ ]:
%%sql
DROP TABLE MovieStar;

In [ ]:
%%sql
CREATE TABLE MovieStar ( 
    name      CHAR(30) PRIMARY KEY, 
    lastname  CHAR(30) UNIQUE,
    address   VARCHAR(255) NOT NULL, 
    gender    CHAR(1),
    birthdate DATE, 
    ASSERTION MovieStarCheck CHECK (gender ='F' OR gender='M')
  );

In [ ]:
%%sql
DROP TABLE MovieStar;

In [ ]:
%%sql
CREATE TABLE MovieStar ( 
    name      CHAR(30) PRIMARY KEY, 
    lastname  CHAR(30) UNIQUE,
    address   VARCHAR(255) NOT NULL, 
    gender    CHAR(1),
    length    INT, 
    CONSTRAINT MovieStarCheck CHECK (gender ='F' OR gender='M')
  );

Why does the following assertion fail?

In [ ]:
%%sql
DROP TABLE Movies;

In [ ]:
%%sql
CREATE TABLE Movies(
    title        CHAR(100),
    year         INT,
    length       INT,
    genre        CHAR(10),
    studioName   CHAR(30),
    producerCNum INT,
    ASSERTION SumLength CHECK (10000 >= 
             (SELECT SUM(length) 
              FROM Movies )), 
    PRIMARY KEY  (title, year)
);

**Note:** SQLite does not support subqueries in assertions as well as altering tables after creation. Either of these restrictions have an effect on constraints and triggers. 

## Triggers
Note that the trigger syntax in SQLite is a bit different. You can find out more about it [here](https://www.sqlitetutorial.net/sqlite-trigger/).

In [ ]:
%%sql
CREATE TRIGGER NetWorthTrigger
AFTER UPDATE OF netWorth ON MovieExec 
WHEN (old.netWorth > new.netWorth) 
    BEGIN
    UPDATE MovieExec SET netWorth = old.netWorth
    WHERE certNum = old.certNum; 
    END;


To detele the trigger, you can use 'DROP TRIGGER':

In [ ]:
%%sql
DROP TRIGGER NetWorthTrigger;

### Clean up Steps

In [ ]:
%%sql
DROP TABLE Enrolled;
DROP TABLE Students;
DROP TABLE Movies;
DROP TABLE MovieStar;
DROP TABLE MovieExec;
DROP TABLE StarsIn;